In [1]:
#Use findspark to confirm pyspark works
import findspark
import streamlit
findspark.init()

In [2]:
#Import pyspark
import pyspark
from pyspark import SparkContext, SparkConf
from collections import Counter
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split, col

from pyspark.sql import SparkSession
#spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.master('local[10]').config('spark.driver.memory','10g').getOrCreate()

conf = SparkConf().setAppName('Project').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf) #SparkContext(conf = conf)

/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 12:09:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in and assign column names
# Income data
Income = spark.read.csv('income_csv_cleaned_no_commas.csv',header=False)
Income = Income.toDF('Area_code','Area','Self_num_ind','Self_mean','Self_median','Num_ind','Mean','Median','Pens_num_ind','Pens_mean',
                     'Pens_median','Other_num_ind','Other_mean','Other_median','Tot_num_ind','Tot_mean','Tot_median','Tax_num_ind','Tax_mean','Tax_median','Tot_tax')

# Drop not applicable
Income = Income.na.drop()

# Remove original formatting
Income = Income.filter(Income.Area!='Region/County')

# Population data
Population = spark.read.csv('population_data.csv',header=False)
Population = Population.toDF('Code','Name','Geography','All ages','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
                     '16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37',
                     '38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59',
                     '60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80','81','82','83','84','85','86','87','88','89','90+')

# Drop not applicable
Population = Population.na.drop()

# Remove original formatting
Population = Population.filter(Population.Code!='Code')

# Income data
Statements = spark.read.csv('Transposed_Consolidated_Income_Statements.csv',header=False)
Statements = Statements.toDF('Company','Fina_period','State','Revenue','Cost','Gross','Other','Disp_close','Admin','Operating','Fin_out','Fin_in','Share_loss','Before_tax','Tax','Profit')

# Remove original formatting
Statements = Statements.drop(col('Company'))
Statements = Statements.drop(col('Fina_period'))
Statements = Statements.drop(col('State'))

# Remove original formatting
Statements = Statements.filter(Statements.Cost!='Cost of sales')

Income.show()

+--------------+--------------------+------------+---------+-----------+-------+-----+------+------------+---------+-----------+-------------+----------+------------+-----------+--------+----------+-----------+--------+----------+-------+
|     Area_code|                Area|Self_num_ind|Self_mean|Self_median|Num_ind| Mean|Median|Pens_num_ind|Pens_mean|Pens_median|Other_num_ind|Other_mean|Other_median|Tot_num_ind|Tot_mean|Tot_median|Tax_num_ind|Tax_mean|Tax_median|Tot_tax|
+--------------+--------------------+------------+---------+-----------+-------+-----+------+------------+---------+-----------+-------------+----------+------------+-----------+--------+----------+-----------+--------+----------+-------+
|     K02000001|      United Kingdom|        3520|    30300|      16300|  25000|36800| 26500|        8560|    19900|      16900|        14200|      7240|          60|      33000|   39400|     27200|      33000|    6850|      2620| 226000|
|     E92000001|             England|       

In [4]:
import numpy as np

# Condensing the data to only what is relevant
# Dropping self employement data

# Income condensed
Cond_Inc = Income

Cond_Inc = Cond_Inc.drop(col('Self_num_ind'))
Cond_Inc = Cond_Inc.drop(col('Self_mean'))
Cond_Inc = Cond_Inc.drop(col('Self_median'))

# Dropping pension data
Cond_Inc = Cond_Inc.drop(col('Pens_num_ind'))
Cond_Inc = Cond_Inc.drop(col('Pens_mean'))
Cond_Inc = Cond_Inc.drop(col('Pens_median'))

# Remove the non-applicable areas
Cond_Inc = Cond_Inc.filter(Cond_Inc.Area_code != 'Not applicable')

# Population condensed
Cond_Pop = Population

# Dropping columns for children
for colu in range(0,16):
    Cond_Pop = Cond_Pop.drop('',col(str(colu)))

# Dropping repeated data in both population and income
Cond_Pop = Cond_Pop.drop(col('Geography'))
Cond_Pop = Cond_Pop.drop(col('Name'))

# Remove commas from numbers
#Cond_Pop = Cond_Pop.map(lambda x: str(x).replace(',', '') if isinstance(x, str) else x)

# Join two dataframes
Data = Cond_Inc.join(Cond_Pop,Cond_Inc.Area_code == Cond_Pop.Code,'inner')
Data = Data.drop(col('Area_code'))

# Get all codes in the new dataframe
codes = [row.Code for row in Data.select('Code').collect()]

Data = Data.withColumn("Num_ind",Data['Num_ind'].cast(IntegerType()))

for col in Data.dtypes:
    print(col[0]+" , "+col[1])



25/03/05 12:09:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


IntegerType()
Area , string
Num_ind , int
Mean , string
Median , string
Other_num_ind , string
Other_mean , string
Other_median , string
Tot_num_ind , string
Tot_mean , string
Tot_median , string
Tax_num_ind , string
Tax_mean , string
Tax_median , string
Tot_tax , string
Code , string
All ages , string
16 , string
17 , string
18 , string
19 , string
20 , string
21 , string
22 , string
23 , string
24 , string
25 , string
26 , string
27 , string
28 , string
29 , string
30 , string
31 , string
32 , string
33 , string
34 , string
35 , string
36 , string
37 , string
38 , string
39 , string
40 , string
41 , string
42 , string
43 , string
44 , string
45 , string
46 , string
47 , string
48 , string
49 , string
50 , string
51 , string
52 , string
53 , string
54 , string
55 , string
56 , string
57 , string
58 , string
59 , string
60 , string
61 , string
62 , string
63 , string
64 , string
65 , string
66 , string
67 , string
68 , string
69 , string
70 , string
71 , string
72 , string
73 , string


In [5]:
'''
# Imports
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.recommendation import ALS

# Split data
train_data, test_data = Data.randomSplit([0.8, 0.2])

# Build recommender model
als = ALS(maxIter=5, regParam=0.01, userCol='Truster', itemCol='Movie', ratingCol='Rating') 

# Fit
model = als.fit(train_data)

# Evaluate
predictions = model.transform(test_data)
predictions.show()

# Drop null
predictions = predictions.na.drop() 

# RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating",predictionCol="prediction") 
rmse = evaluator.evaluate(predictions) 
print("Root-mean-square error = " + str(rmse))
'''

'\n# Imports\nfrom pyspark.ml.evaluation import RegressionEvaluator \nfrom pyspark.ml.recommendation import ALS\n\n# Split data\ntrain_data, test_data = Data.randomSplit([0.8, 0.2])\n\n# Build recommender model\nals = ALS(maxIter=5, regParam=0.01, userCol=\'Truster\', itemCol=\'Movie\', ratingCol=\'Rating\') \n\n# Fit\nmodel = als.fit(train_data)\n\n# Evaluate\npredictions = model.transform(test_data)\npredictions.show()\n\n# Drop null\npredictions = predictions.na.drop() \n\n# RMSE\nevaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating",predictionCol="prediction") \nrmse = evaluator.evaluate(predictions) \nprint("Root-mean-square error = " + str(rmse))\n'